# Generate Test Data

In [ ]:
import xarray as xr
import numpy as np
import dask
import intake
from getpass import getuser
from pathlib import Path

import marEx
import marEx.helper as hpc

In [ ]:
# Lustre Scratch Directory
scratch_dir = Path('/scratch') / getuser()[0] / getuser()
save_dir = Path('/home') / getuser()[0] / getuser() / 'opt' / 'marEx' / 'tests' / 'data'

In [ ]:
# Start Dask Cluster
client = hpc.start_local_cluster(n_workers=32, threads_per_worker=1,
                                 scratch_dir = scratch_dir / 'clients')  # Specify temporary scratch directory for dask to use

# Raw SST Data

In [ ]:
# Import 40 years of Daily ICON data (ref. EERIE project)

cat = intake.open_catalog("https://raw.githubusercontent.com/eerie-project/intake_catalogues/main/eerie.yaml")
expid = 'eerie-control-1950'
version = 'v20240618'
model = 'icon-esm-er'

dat_regrid = cat['dkrz.disk.model-output'][model][expid][version]['ocean']['gr025']
dat_native = cat['dkrz.disk.model-output'][model][expid][version]['ocean']['native']

In [ ]:
# Load Data
sst_regrid = dat_regrid['2d_daily_mean'](chunks={}).to_dask().to.isel(depth=0).drop_vars('depth')
sst_native = dat_native['2d_daily_mean'](chunks={}).to_dask().to.isel(depth=0).drop_vars({'depth','cell_sea_land_mask'}).chunk({'time':32})

In [ ]:
# Define subset
time_slice = slice('1991-01-01', '2030-01-01')
regrid_slice = dict(lat=slice(35, 40), lon=slice(320, 330))
native_slice = slice(0,1000)

In [ ]:
sst_regrid_slice = sst_regrid.sel(time=time_slice, **regrid_slice)
sst_native_slice = sst_native.sel(time=time_slice).isel(ncells=native_slice).drop_vars({'lat','lon'})

In [ ]:
sst_regrid_slice

In [ ]:
sst_native_slice

In [ ]:
# Save to Zarr
sst_regrid_slice.to_zarr(save_dir / 'sst_gridded.zarr', mode='w')
sst_native_slice.to_zarr(save_dir / 'sst_unstructured.zarr', mode='w')

# Pre-processed Data

In [ ]:
time_slice = slice('2010-01-01', '2010-02-01')
extremes_ds = xr.open_zarr(scratch_dir / 'mhws' / 'extremes_binary_gridded_shifting.zarr', chunks={}).sel(time=time_slice).drop_vars({'thresholds','dat_detrend','dayofyear'})

In [ ]:
extremes_ds

In [ ]:
# Coarsen extremes_ds (lat & lon)
extremes_ds_coarsen = extremes_ds.coarsen(lat=4, lon=4, boundary='trim').any().chunk({'time': 2, 'lat':-1, 'lon':-1}).persist()
extremes_ds_coarsen

In [ ]:
extremes_ds_coarsen = extremes_ds_coarsen.compute()
extremes_ds_coarsen = extremes_ds_coarsen.chunk({'time':2, 'lat':-1, 'lon':-1})
# Clear encoding
extremes_ds_coarsen.encoding = {}
for var in extremes_ds_coarsen.data_vars:
	extremes_ds_coarsen[var].encoding = {}

In [ ]:
extremes_ds_coarsen.to_zarr(save_dir / 'extremes_gridded.zarr', mode='w')